In [1]:
import requests
import pandas as pd

In [2]:
from bs4 import BeautifulSoup

In [3]:
#variable tahun
years = list(range(2022,2021,-1))

In [4]:
#variable kosong untuk menampung hasil scrapping
all_matches=[]

In [5]:
standings_url ="https://fbref.com/en/comps/9/Premier-League-Stats"

In [6]:
for year in years:
  data= requests.get(standings_url)
  soup=BeautifulSoup(data.text)
  standings_table = soup.select('table.stats_table')[0]
  links = [l.get("href") for l in standings_table.find_all('a')]
  links = [l for l in links if'/squads/' in l]
  team_urls = [f"https://fbref.com{l}" for l in links]

In [7]:
team_urls

['https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats',
 'https://fbre

In [8]:
import time

for year in years:
  data= requests.get(standings_url)
  soup = BeautifulSoup(data.text)
  standings_table = soup.select('table.stats_table')[0]

  links = [l.get("href") for l in standings_table.find_all('a')]
  links = [l for l in links if '/squads/' in l]
  team_urls = [f"https://fbref.com{l}" for l in links]

  for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats","").replace("-"," ")

    data = requests.get(team_url)
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

    soup = BeautifulSoup(data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and '/all_comps/shooting' in l]
    links = f"https://fbref.com{links[0]}"
    data1 = requests.get(links)
    shooting = pd.read_html(data1.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()

    try:
      team_data = matches.merge(shooting[["Date","Sh","SoT","Dist","FK","PK","PKatt"]],on="Date")
    except ValueError:
      continue
    team_data = team_data[team_data["Comp"] == "Premier League"]
    team_data["Season"] = year
    team_data["Team"] = team_name
    all_matches.append(team_data)
    time.sleep(3)

In [9]:
len(all_matches)

20

In [10]:
match_df = pd.concat(all_matches)

In [11]:
match_df.columns = [c.lower() for c in match_df.columns]

In [12]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2023-08-13,16:30,Premier League,Matchweek 1,Sun,Away,D,1.0,1.0,Chelsea,...,Match Report,NaN,13.0,1.0,17.8,0.0,0,0,2022,Liverpool
1,2023-08-19,15:00,Premier League,Matchweek 2,Sat,Home,W,3.0,1.0,Bournemouth,...,Match Report,NaN,25.0,9.0,16.8,1.0,0,1,2022,Liverpool
2,2023-08-27,16:30,Premier League,Matchweek 3,Sun,Away,W,2.0,1.0,Newcastle Utd,...,Match Report,NaN,9.0,4.0,17.2,1.0,0,0,2022,Liverpool
3,2023-09-03,14:00,Premier League,Matchweek 4,Sun,Home,W,3.0,0.0,Aston Villa,...,Match Report,NaN,17.0,4.0,14.7,0.0,0,0,2022,Liverpool
4,2023-09-16,12:30,Premier League,Matchweek 5,Sat,Away,W,3.0,1.0,Wolves,...,Match Report,NaN,16.0,5.0,15.8,0.0,0,0,2022,Liverpool
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,2023-12-30,15:00,Premier League,Matchweek 20,Sat,Away,L,0,2,Manchester City,...,Match Report,NaN,4.0,2.0,14.9,0.0,0,0,2022,Sheffield United
22,2024-01-21,14:00,Premier League,Matchweek 21,Sun,Home,D,2,2,West Ham,...,Match Report,NaN,20.0,5.0,18.5,1.0,1,1,2022,Sheffield United
24,2024-01-30,20:00,Premier League,Matchweek 22,Tue,Away,L,2,3,Crystal Palace,...,Match Report,NaN,9.0,5.0,16.5,0.0,0,0,2022,Sheffield United
25,2024-02-03,17:30,Premier League,Matchweek 23,Sat,Home,L,0,5,Aston Villa,...,Match Report,NaN,10.0,4.0,10.0,0.0,0,0,2022,Sheffield United


In [13]:
#import menjadi csv file
match_df.to_csv("matches.csv")